# Hyperparameter Tuning

* Random Forest


## I. Import libs and data

In [13]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import SMOTE
import pickle
from sklearn.metrics import roc_auc_score

from lightgbm.sklearn import LGBMClassifier

### Import Data

In [2]:
data_train = pd.read_csv("cleaned_2013_14", low_memory = False);
data_test = pd.read_csv("cleaned_2015", low_memory = False);

In [3]:
data_train.columns

Index(['funded_amnt', 'int_rate', 'total_pymnt', 'annual_inc', 'dti',
       'loan_status', 'revol_util', 'term', 'term_adj', 'zip_code',
       'emp_length_1 year', 'emp_length_10+ years', 'emp_length_2 years',
       'emp_length_3 years', 'emp_length_4 years', 'emp_length_5 years',
       'emp_length_6 years', 'emp_length_7 years', 'emp_length_8 years',
       'emp_length_9 years', 'emp_length_< 1 year', 'home_ownership_ANY',
       'home_ownership_MORTGAGE', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'purpose_car', 'purpose_credit_card',
       'purpose_debt_consolidation', 'purpose_home_improvement',
       'purpose_house', 'purpose_major_purchase', 'purpose_medical',
       'purpose_moving', 'purpose_other', 'purpose_renewable_energy',
       'purpose_small_business', 'p

In [4]:
data_test = data_test.dropna()
data_test.columns == data_train.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [5]:
def split_data(df, cols):
    x = df.drop(cols, axis = 1)
    y = df.paid
    return x, y

cols_to_drop_training = ['loan_status', 'paid', 'amnt', 'total_pymnt', 'term_adj', 'zip_code']
x_train_initial, y_train_initial = split_data(data_train, cols_to_drop_training)
x_test, y_test = split_data(data_test, cols_to_drop_training)

In [6]:
sm = SMOTE(random_state=1, ratio = 1.0)
x_train, y_train = sm.fit_sample(x_train_initial, y_train_initial)

In [7]:
x_train.shape, x_test.shape

((578050, 44), (346843, 44))

### Downsample

In [8]:
## Downsample the data to 30% of the full dataset stratifying by classes
dont_use_x, x_train_sample, dont_use_y , y_train_sample = train_test_split(x_train, 
                                                            y_train, test_size = 0.3, stratify=y_train)

In [9]:
x_train_sample.shape, y_train_sample.shape

((173415, 44), (173415,))

## Random Forest Tuning

In [12]:
# Use Pipeline instead of make_pipeline to do grid search
pipe = Pipeline([('POLY', PolynomialFeatures(degree=2, include_bias=False)),
            ('RFC',RandomForestClassifier(max_features = 100))])

# Specify parameters 
param_grid = {'RFC__n_estimators' : [50,100,120,140],
              'RFC__max_depth' : [8,10,12]}

# Instantiate GS
gs = GridSearchCV(pipe,param_grid, verbose=10, cv=4)

In [13]:
gs.fit(x_train_sample, y_train_sample)

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] RFC__max_depth=8, RFC__n_estimators=50 ..........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  RFC__max_depth=8, RFC__n_estimators=50, score=0.6933154956866725, total= 1.9min
[CV] RFC__max_depth=8, RFC__n_estimators=50 ..........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s


[CV]  RFC__max_depth=8, RFC__n_estimators=50, score=0.6937306822899848, total= 1.8min
[CV] RFC__max_depth=8, RFC__n_estimators=50 ..........................


...

[CV]  RFC__max_depth=8, RFC__n_estimators=120, score=0.6956451538497025, total= 3.8min
[CV] RFC__max_depth=8, RFC__n_estimators=120 .........................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 26.2min remaining:    0.0s


[CV]  RFC__max_depth=8, RFC__n_estimators=120, score=0.6922544632559856, total= 3.6min
[CV] RFC__max_depth=8, RFC__n_estimators=120 .........................
[CV]  RFC__max_depth=8, RFC__n_estimators=120, score=0.6956220879272962, total= 3.6min
[CV] RFC__max_depth=8, RFC__n_estimators=120 .........................
[CV]  RFC__max_depth=8, RFC__n_estimators=120, score=0.6943002791040989, total= 3.6min
[CV] RFC__max_depth=8, RFC__n_estimators=140 .........................
[CV]  RFC__max_depth=8, RFC__n_estimators=140, score=0.6953683627808276, total= 4.5min
[CV] RFC__max_depth=8, RFC__n_estimators=140 .........................
[CV]  RFC__max_depth=8, RFC__n_estimators=140, score=0.6893250911103935, total= 4.2min
[CV] RFC__max_depth=8, RFC__n_estimators=140 .........................
[CV]  RFC__max_depth=8, RFC__n_estimators=140, score=0.6957374175393274, total= 4.2min
[CV] RFC__max_depth=8, RFC__n_estimators=140 .........................
[CV]  RFC__max_depth=8, RFC__n_estimators=140, score

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 186.7min finished


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('POLY', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('RFC', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=100, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'RFC__n_estimators': [50, 100, 120, 140], 'RFC__max_depth': [8, 10, 12]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [14]:
print("Best parameters set found on training set:")
print()
print(gs.best_params_)
print()
print("Grid scores on training set:")
print()
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

Best parameters set found on training set:

{'RFC__max_depth': 12, 'RFC__n_estimators': 50}

Grid scores on training set:

0.694 (+/-0.002) for {'RFC__max_depth': 8, 'RFC__n_estimators': 50}
0.694 (+/-0.001) for {'RFC__max_depth': 8, 'RFC__n_estimators': 100}
0.694 (+/-0.003) for {'RFC__max_depth': 8, 'RFC__n_estimators': 120}
0.694 (+/-0.005) for {'RFC__max_depth': 8, 'RFC__n_estimators': 140}
0.713 (+/-0.005) for {'RFC__max_depth': 10, 'RFC__n_estimators': 50}
0.715 (+/-0.003) for {'RFC__max_depth': 10, 'RFC__n_estimators': 100}
0.714 (+/-0.004) for {'RFC__max_depth': 10, 'RFC__n_estimators': 120}
0.714 (+/-0.007) for {'RFC__max_depth': 10, 'RFC__n_estimators': 140}
0.739 (+/-0.005) for {'RFC__max_depth': 12, 'RFC__n_estimators': 50}
0.737 (+/-0.005) for {'RFC__max_depth': 12, 'RFC__n_estimators': 100}
0.736 (+/-0.007) for {'RFC__max_depth': 12, 'RFC__n_estimators': 120}
0.737 (+/-0.008) for {'RFC__max_depth': 12, 'RFC__n_estimators': 140}



In [15]:
samp_train_preds_true = {'proba' : gs.best_estimator_.predict_proba(x_train_sample)[:, 1], 'true_val' : y_train_sample}
train_preds_true = {'proba' : gs.best_estimator_.predict_proba(x_train)[:, 1], 'true_val' : y_train}
test_preds_true = {'proba' : gs.best_estimator_.predict_proba(x_test)[:, 1], 'true_val' : y_test}

In [18]:
AUC_sample = roc_auc_score(samp_train_preds_true['true_val'], samp_train_preds_true['proba'])
AUC_train = roc_auc_score(train_preds_true['true_val'], train_preds_true['proba'])
AUC_test = roc_auc_score(test_preds_true['true_val'], test_preds_true['proba'])

In [20]:
print('AUC Sample: ', AUC_sample);
print('AUC Train: ', AUC_train);
print('AUC Test: ', AUC_test);

AUC Sample:  0.847602145427717
AUC Train:  0.8254550459486434
AUC Test:  0.637394906801299


In [21]:
# Dump the trained decision tree classifier with Pickle
decision_tree_pkl_filename = 'Tuned_RF.pkl'
# Open the file to save as pkl file
decision_tree_model_pkl = open(decision_tree_pkl_filename, 'wb')
pickle.dump(gs.best_estimator_, decision_tree_model_pkl)
# Close the pickle instances
decision_tree_model_pkl.close()

##  LGBM Tuning

In [26]:
lgbm_mod = LGBMClassifier()

# Specify parameters 
gridParams = {
    'learning_rate': [0.005],
    'n_estimators': [40],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

# Instantiate GS
gs = GridSearchCV(lgbm_mod, gridParams, verbose=10, cv=4)

In [27]:
gs.fit(x_train_sample, y_train_sample)

Fitting 4 folds for each of 96 candidates, totalling 384 fits
[CV] boosting_type=gbdt, colsample_bytree=0.65, learning_rate=0.005, n_estimators=40, num_leaves=6, objective=binary, reg_alpha=1, reg_lambda=1, subsample=0.7 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  boosting_type=gbdt, colsample_bytree=0.65, learning_rate=0.005, n_estimators=40, num_leaves=6, objective=binary, reg_alpha=1, reg_lambda=1, subsample=0.7, score=0.6572173271209115, total=   0.6s
[CV] boosting_type=gbdt, colsample_bytree=0.65, learning_rate=0.005, n_estimators=40, num_leaves=6, objective=binary, reg_alpha=1, reg_lambda=1, subsample=0.7 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  boosting_type=gbdt, colsample_bytree=0.65, learning_rate=0.005, n_estimators=40, num_leaves=6, objective=binary, reg_alpha=1, reg_lambda=1, subsample=0.7, score=0.6607002814042534, total=   0.5s
[CV] boosting_type=gbdt, colsample_bytree=0.65, learning_rate=0.005, n_estimators=40, num_leaves=6, objective=binary, reg_alpha=1, reg_lambda=1, subsample=0.7 


...

[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=12, objective=binary, reg_alpha=1.2, reg_lambda=1.4, subsample=0.7, score=0.6949461398288469, total=   0.5s
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=12, objective=binary, reg_alpha=1.2, reg_lambda=1.4, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=12, objective=binary, reg_alpha=1.2, reg_lambda=1.4, subsample=0.75, score=0.6760160538819947, total=   0.5s
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=12, objective=binary, reg_alpha=1.2, reg_lambda=1.4, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=12, objective=binary, reg_alpha=1.2, reg_lambda=1.4, subsample=0.75, score=0.6801909858375237, total=   0.5s
[CV] boosting_type=gbdt, colsample_bytree=0.66, 

[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=16, objective=binary, reg_alpha=1, reg_lambda=1.4, subsample=0.7, score=0.7001891405637312, total=   0.5s
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=16, objective=binary, reg_alpha=1, reg_lambda=1.4, subsample=0.7 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=16, objective=binary, reg_alpha=1, reg_lambda=1.4, subsample=0.7, score=0.7021659400733513, total=   0.5s
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=16, objective=binary, reg_alpha=1, reg_lambda=1.4, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=16, objective=binary, reg_alpha=1, reg_lambda=1.4, subsample=0.75, score=0.6849425658532085, total=   0.5s
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rat

[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=16, objective=binary, reg_alpha=1.2, reg_lambda=1.4, subsample=0.7, score=0.6894404207224247, total=   0.5s
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=16, objective=binary, reg_alpha=1.2, reg_lambda=1.4, subsample=0.7 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=16, objective=binary, reg_alpha=1.2, reg_lambda=1.4, subsample=0.7, score=0.7003044701757624, total=   0.5s
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=16, objective=binary, reg_alpha=1.2, reg_lambda=1.4, subsample=0.7 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.005, n_estimators=40, num_leaves=16, objective=binary, reg_alpha=1.2, reg_lambda=1.4, subsample=0.7, score=0.7031347311604733, total=   0.5s
[CV] boosting_type=gbdt, colsample_bytree=0.66, lear

[Parallel(n_jobs=1)]: Done 384 out of 384 | elapsed:  3.9min finished


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.005], 'n_estimators': [40], 'num_leaves': [6, 8, 12, 16], 'boosting_type': ['gbdt'], 'objective': ['binary'], 'colsample_bytree': [0.65, 0.66], 'subsample': [0.7, 0.75], 'reg_alpha': [1, 1.2], 'reg_lambda': [1, 1.2, 1.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)